### Creating and Organizing Dataframe

In [1]:
import pandas as pd
import numpy as np
import requests
import xml.etree.ElementTree as ET

df = pd.read_csv("beacon.csv")
df = df[['filename', 'number', 'mods.physicalDescriptionextent', 'pageTitle', 'mods.titleInfo.0title']]	

# page title to string
df['pageTitle'] = df['pageTitle'].astype('str') 
df['pageTitle'] = df['pageTitle'].str.replace('.0', '')

# num of pages to int
df['mods.physicalDescriptionextent'] = df['mods.physicalDescriptionextent'].str.replace(' pages', '')
df['mods.physicalDescriptionextent'] = pd.to_numeric(df['mods.physicalDescriptionextent'], errors='coerce')

# sort by pid number
df = df.sort_values('number', ascending = True)

df.head(1000)

C:\Users\aquijano\AppData\Local\Temp\ipykernel_9308\2374902646.py:6: DtypeWarning: Columns (13,14,15,42,43,44,45,48,56,57) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("beacon.csv")


,filename,number,mods.physicalDescriptionextent,pageTitle,mods.titleInfo.0title
16619,beacon:1,1,6.0,nan,"The daily beacon, October 9, 1968"
22897,beacon:2,2,NaN,6,NaN
20310,beacon:3,3,NaN,5,NaN
10674,beacon:4,4,NaN,4,NaN
7945,beacon:5,5,NaN,3,NaN
...,...,...,...,...,...
21242,beacon:1010,1010,NaN,10,NaN
22078,beacon:1011,1011,NaN,9,NaN
15920,beacon:1012,1012,NaN,8,NaN
14892,beacon:1013,1013,NaN,7,NaN


### Find Missing PIDs

In [9]:
pids = df['number']
missing_pids = np.setdiff1d(np.arange(pids.min(), pids.max()+1), pids)

print(np.count_nonzero(missing_pids))

11519


### Add Missing Rows

In [ ]:
def get_title_from_pid(pid):
    url = f"https://digital.lib.utk.edu/collections/islandora/object/beacon%3A{pid}/datastream/MODS/view"
    
    try:
        # Fetch the XML data from the URL
        response = requests.get(url)
        response.raise_for_status()  # Check if the request was successful

        # Parse the XML data
        root = ET.fromstring(response.content)

        # Define the namespace
        namespaces = {
            'mods': 'http://www.loc.gov/mods/v3'
        }

        # Find the title within the XML structure
        title_element = root.find('.//mods:titleInfo/mods:title', namespaces)

        # Extract and return the title
        if title_element is not None:
            return title_element.text
        else:
            return None
    except requests.exceptions.RequestException as e:
        return None
        
for pid in missing_pids:
    title = get_title_from_pid(pid)
    if title is not None:
        data.append({pid, }, ignore_index = True)
        print(f"PID: {pid} -> Title: {title}")     